## Sales Forecasting with Bayesian Inference

This notebook demonstrates the use of Bayesian inference for sales forecasting using various probabilistic programming techniques. We will use the `numpyro` library to define and fit our models, and `plotnine` for visualization.

## Introduction

Bayesian inference allows us to incorporate prior knowledge and quantify uncertainty in our predictions. This notebook will guide you through the process of building a Bayesian model for sales forecasting, fitting the model using Markov Chain Monte Carlo (MCMC) and Stochastic Variational Inference (SVI), and visualizing the results.

## Import Required Libraries And Define Functions

In [19]:
import os
import sys

# Set XLA_FLAGS before JAX is imported
os.environ["XLA_FLAGS"] = "--xla_force_host_platform_device_count=8"

In [ ]:
import polars as pl
#import numpy as np
#import pandas as pd
from plotnine import ggplot, aes, geom_point, geom_line, labs, theme_minimal, theme_bw, scale_x_continuous, scale_x_discrete, scale_x_datetime
import patsy

import jax.numpy as jnp
import jax.random as random

import numpyro
import numpyro.distributions as dist
#from numpyro.infer import SVI, Trace_ELBO, Predictive
from numpyro.infer import MCMC, NUTS, MCMC, NUTS #, SVI, Trace_ELBO
import arviz as az


## 1. Model

In this section, we define the Bayesian model used for sales forecasting. The model incorporates various components such as random walk for the latent state, day-of-the-week effects, day-of-the-year effects, and price elasticity. The model is implemented using the `numpyro` library, which allows for efficient and scalable Bayesian inference.

### 1.1 Auxiliary Functions

These auxiliary functions are essential for data preprocessing and transformation:

- `periodic_rbf`: Computes a periodic Gaussian radial basis function (RBF).
- `compute_doy_basis`: Computes 12 periodic Gaussian basis functions for seasonal effects.
- `read_data`: Reads and preprocesses the sales data from a CSV file.
- `init_values`: Initializes values for the model parameters.

In [ ]:
# Define a periodic Gaussian radial basis function (RBF)
def periodic_rbf(x, mu, sigma):
    """
    Computes a periodic Gaussian radial basis function (RBF).
    
    Args:
        x: Scaled day-of-year values (range [0,1]).
        mu: Center of the Gaussian basis function.
        sigma: Controls the spread of the Gaussian.
    
    Returns:
        RBF values preserving periodicity.
    """
    # compute cyclic distance to mu
    periodic_distance = jnp.minimum(jnp.abs(x - mu), 1 - jnp.abs(x - mu))
    # compute RBF value
    return jnp.exp(- (periodic_distance ** 2) / (2 * sigma ** 2))

def compute_doy_basis(yday_fraction, sigma = 30/365.25, n_centers = 12):
    """
    Computes 12 periodic Gaussian basis functions for seasonal effects.
    
    Args:
        yday_fraction: Normalized day of the year (range [0,1]).
        yday_factor: Scaling factor for basis function width.
    
    Returns:
        A JAX array with 12 columns representing the 12 monthly basis functions.
    """
    # Define centers of Gaussian basis functions
    month_centers = jnp.linspace( 1/(2*n_centers), 1-1/(2*n_centers), n_centers)
    
    # Generate an array of shape (length of input, 12) with the RBF values
    doy_basis = jnp.stack([periodic_rbf(yday_fraction, mu, sigma) for mu in month_centers], axis=-1)

    # Subtract each row's mean to enforce sum-to-zero constraint
    doy_basis_centered = doy_basis - jnp.mean(doy_basis, axis=-1, keepdims=True)
    
    return doy_basis_centered

def read_data(fname):
    """
    Reads and preprocesses the sales data from a CSV file.
    
    Args:
        fname: The filename of the CSV file containing the sales data.
    
    Returns:
        A dictionary with the following keys:
            - sales: An array of sales data.
            - log_price: An array of log-transformed prices.
            - wday: An array of day-of-the-week values.
            - yday_fraction: An array of normalized day-of-the-year values.
    """
    # Read the CSV file using polars
    df = pl.read_csv(fname)
    
    # Convert the 'date' column to date type
    df = df.with_columns(pl.col("date").str.to_date())

    # Extract sales, and log price data as a numpy arrays
    sales = df["sales"].to_numpy()
    log_price = df["log_price"].to_numpy()
    
    # Extract day-of-the-week values
    wday = df["date"].dt.weekday().to_numpy()
    
    # Extract day-of-the-year values
    yday = df["date"].dt.ordinal_day().to_numpy()
    
    # Determine if the year is a leap year
    is_leap_year = df["date"].dt.is_leap_year().to_numpy()
    
    # Normalize day-of-the-year values
    yday_fraction = yday / (365 + is_leap_year)
    
    # Return the preprocessed data as a dictionary
    return {
        "sales": sales,
        "log_price": log_price,
        "wday": wday,
        "yday_fraction": yday_fraction
    }

#def init_values(sales: jnp.array, log_price_centered: jnp.array, wday, yday_fraction: jnp.array, downsampling_factor = 1):
#    """
#    """
#    # to-do: implement downsampling_factor
#    log_state_est = jnp.log(sales)
#    log_state_mean_est = jnp.mean(log_state_est)
#    log_state_delta_est = jnp.diff(log_state_est)
#    log_state_delta_sd_est = jnp.std(log_state_delta_est)
#
#    return {
#        "log_sigma": jnp.log( log_state_delta_sd_est ),
#        "log_state_mean": log_state_mean_est,
#        "log_state_delta": log_state_delta_est,
#        "wday_coefficients": jnp.array([0.0]*6),
#        "yday_coefficients": jnp.array([0.0]*12),
#        "log_elasticity": jnp.array([0.0])
#    }

### 1.2 Model

#### 1.2.1 Key Features

- Sales are modeled using a **stochastic Poisson process**, where the expected rate $\lambda_t$ evolves over time.
- The **latent sales rate** follows a random walk, allowing it to drift nonstationarily.
- **Seasonal components** (day-of-the-week and annual patterns) adjust for structured demand variations.
- **Price elasticity** is explicitly modeled, ensuring sensitivity to pricing dynamics.
- The model is implemented in numpyro, enabling scalable Bayesian inference.


#### 1.2.1 Model Overview

We model the sales time series as a **stochastic process** where the underlying rate of sales evolves over time. This evolution follows a **random walk structure**, but with systematic adjustments for covariates such as price, day-of-the-week effects, and day-of-the-year effects. The rate of sales $\lambda_t$ on day $t$ is a function of captures *(i)* systematic covariate effects ($z_t$), *(ii)*
a global baseline ($\mu_\tau$), and *(iii)* the latent dynamic component ($\tau_t$).

$$
log~\lambda_t = z_t + \mu_\tau + \tau_t
$$

##### 1.2.1.1 Latent States Dynamics

The baseline sales level $\tau_t$ follows a **random walk**. Because all contrast matrices for structured effects are centered, $\mu_\tau + \tau_t$ can be interpreted as the average latent sales rate on $\tau_t$. 

$$
\tau_t = \tau_{t-1} + \delta_t, \quad \delta_t \sim \mathcal{N}(0, \sigma_\tau)
$$

with:

$$
\mu_\tau \sim \text{Exponential}(1), \quad \sigma_\tau \sim \mathcal{N}(1)
$$


##### 1.2.1.2 Structured Effects

We further accounted for systematic effects of *(i)* day of the week, *(ii)* day of the year, and *(iii)* price.

- For day of the week effects, we used a contrast matrix $\mathbf{C}_{\text{wday}}$ with sliding differences.
- For day of the year effects, we used a matrix of Gaussian radial basis functions $\mathbf{B}_{\text{yday}}$.
- Price elasticity is modelled using a centered log price 

Similarly, the day-of-the-year effects are modeled using a seasonality basis matrix $\mathbf{B}_{\text{yday}}$, which represents periodic seasonal patterns using Gaussian radial basis functions (RBFs).


- **Day-of-the-week effects**:

$$
  zw_t = \mathbf{C}_{\text{wday}} \cdot \beta_{\text{wday}}, \quad \beta_{\text{wday}} \sim \mathcal{N}(0, 1)
$$

- **Day-of-the-year effects**:

$$
  zy_t = \mathbf{B}_{\text{yday}} \cdot \beta_{\text{yday}}, \quad \beta_{\text{yday}} \sim \mathcal{N}(0, 1)
$$

- **Price elasticity**:

$$
  ze_t = \text{log\_price\_centered} \cdot e, \quad \log(-e) \sim \mathcal{N^{+}}(0, 1)
$$

- **Sum of structural effects**:

$$
  z_t = zw_t + zy_t + ze_t
$$


##### 1.2.1.3 Emissions Model


The final latent state combines all effects:

$$
\text{state}_t = \exp(\tau_t + w_t + y_t + e_t)
$$


Observed sales follow a **Poisson distribution**, ensuring discrete, non-negative observations:

$$
X_t \sim \text{Poisson}(\lambda_t)
$$

where:

$$
\lambda_t = \text{state}_t
$$


This formulation balances flexibility (random walk) with structured demand influences (seasonality, price sensitivity), making it suitable for forecasting applications.

----



The latent component $\tau_t$ follows a **random walk**:

$$
\tau_t = \tau_{t-1} + \delta_t
$$



$$

$$

𝜏
𝑡
τ 
t
​
  follows a random walk:

𝜏
𝑡
=
𝜏
𝑡
−
1
+
𝛿
𝑡
τ 
t
​
 =τ 
t−1
​
 +δ 
t
​
 
where the step size 
𝛿
𝑡
δ 
t
​
  is drawn from a normal distribution:

𝛿
𝑡
∼
𝑁
(
0
,
𝜎
𝜏
)
δ 
t
​
 ∼N(0,σ 
τ
​
 )
This structure allows 
𝜏
𝑡
τ 
t
​
  to drift over time, making it suitable for capturing long-term shifts in sales trends (e.g., gradual growth or decline).

We also define priors for the key parameters:

The global baseline level follows an exponential prior:
𝜇
𝜏
∼
Exponential
(
1
)
μ 
τ
​
 ∼Exponential(1)
This ensures that 
𝜇
𝜏
μ 
τ
​
  is non-negative and prevents extreme values.
The random walk step size has a normal prior:
𝜎
𝜏
∼
𝑁
(
1
)
σ 
τ
​
 ∼N(1)
This controls the magnitude of daily fluctuations in 
𝜏
𝑡
τ 
t
​
 .



The sales time series will be modeled as a random walk model with covariate-dependent drift, where $\lambda_t$ is the poisson rate on day $t$, $\tau_t$ is <todo>, and $z_t$ is the effect of covariates.


$$
\delta_t \sim \mathcal{N}(0, \sigma_\tau) 
$$

$$
\mu_\tau \sim \text{Exponential}(1)
$$

$$
\sigma_\tau \sim \mathcal{N}(1)
$$

The model is defined using the `numpyro` library, which allows for efficient and scalable Bayesian inference. At its core, it
incorporates various components such as random walk for the latent state, day-of-the-week effects, day-of-the-year effects, and price elasticity. The model is implemented as follows:


1. **Random Walk for Latent State**:
    \[
    \log(\sigma) \sim \text{Gumbel}(0, 1)
    \]
    \[
    \sigma = \exp(\log(\sigma))
    \]
    \[
    \log(\text{state\_mean}) \sim \text{Normal}(0, 5)
    \]
    \[
    \log(\text{state\_delta}) \sim \text{Normal}(0, 1)
    \]
    \[
    \log(\text{state\_base}) = \text{state\_mean} + \sigma \cdot \text{state\_delta}
    \]

2. **Day-of-the-Week Effects**:
    \[
    \text{wday\_coefficients} \sim \text{Normal}(0, 1)
    \]
    \[
    \text{wday\_effect} = \text{wday\_coefficients} \cdot \text{wday}
    \]

3. **Day-of-the-Year Effects**:
    \[
    \text{yday\_coefficients} \sim \text{Normal}(0, 1)
    \]
    \[
    \text{yday\_effect} = \text{yday\_coefficients} \cdot \text{yday\_fraction}
    \]

4. **Price Elasticity**:
    \[
    \log(\text{elasticity}) \sim \text{Normal}(0, 1)
    \]
    \[
    \text{elasticity} = -\exp(\log(\text{elasticity}))
    \]
    \[
    \text{price\_effect} = \text{log\_price\_centered} \cdot \text{elasticity}
    \]

5. **State Computation**:
    \[
    \text{state} = \exp(\log(\text{state\_base}) + \text{price\_effect} + \text{yday\_effect} + \text{wday\_effect})
    \]

6. **Poisson Emissions**:
    \[
    \text{sales} \sim \text{Poisson}(\text{state})
    \]



$$
\sigma \sim \text{Exponential}(1)
$$

$$
\mu_{\tau} \sim \text{Normal}(0, 5)
$$

$$
\log(\tau) \sim \text{Normal}(\mu_{\tau}, \sigma_{\tau})
$$

$$
\sum_{\tau=1}^{T} \Delta \tau
$$

The model can be described mathematically as follows:

1. **Random Walk for Latent State**:
    \[
    \log(\sigma) \sim \text{Gumbel}(0, 1)
    \]
    \[
    \sigma = \exp(\log(\sigma))
    \]
    \[
    \log(\text{state\_mean}) \sim \text{Normal}(0, 5)
    \]
    \[
    \log(\text{state\_delta}) \sim \text{Normal}(0, 1)
    \]
    \[
    \log(\text{state\_base}) = \text{state\_mean} + \sigma \cdot \text{state\_delta}
    \]

2. **Day-of-the-Week Effects**:
    \[
    \text{wday\_coefficients} \sim \text{Normal}(0, 1)
    \]
    \[
    \text{wday\_effect} = \text{wday\_coefficients} \cdot \text{wday}
    \]

3. **Day-of-the-Year Effects**:
    \[
    \text{yday\_coefficients} \sim \text{Normal}(0, 1)
    \]
    \[
    \text{yday\_effect} = \text{yday\_coefficients} \cdot \text{yday\_fraction}
    \]

4. **Price Elasticity**:
    \[
    \log(\text{elasticity}) \sim \text{Normal}(0, 1)
    \]
    \[
    \text{elasticity} = -\exp(\log(\text{elasticity}))
    \]
    \[
    \text{price\_effect} = \text{log\_price\_centered} \cdot \text{elasticity}
    \]

5. **State Computation**:
    \[
    \text{state} = \exp(\log(\text{state\_base}) + \text{price\_effect} + \text{yday\_effect} + \text{wday\_effect})
    \]

6. **Poisson Emissions**:
    \[
    \text{sales} \sim \text{Poisson}(\text{state})
    \]


In [ ]:
def model_local_level_poisson(sales: jnp.array, log_price_centered: jnp.array, wday, yday_fraction: jnp.array, 
                              contrasts_sdif_t: jnp.array, contrasts_wday: jnp.array, contrasts_yday: jnp.array, 
                              downsampling_factor = 1):
    """
    """

    n_obs = len(sales)
    n_states = contrasts_sdif_t.shape[0]
 
    def sample_random_walk(contrasts_sdif_t, n_states):
        log_sigma = numpyro.sample("log_sigma", dist.Gumbel(0, 1))
        sigma = numpyro.deterministic("sigma", jnp.exp(log_sigma))
        log_state_mean = numpyro.sample("log_state_mean", dist.Normal(0, 5))
        log_state_delta = numpyro.sample( "log_state_delta", dist.Normal(0, 1), sample_shape=(n_states-1,))
        log_state_base = numpyro.deterministic("log_state_base", jnp.dot(contrasts_sdif_t, log_state_delta) * sigma + log_state_mean )
        return log_state_base

    def sample_downsampled_random_walk(contrasts_sdif_t, n_obs, n_states):
        log_state_base_downsampled = sample_random_walk(contrasts_sdif_t, n_states)
        
        idx_n_weight = jnp.array(range(0, n_obs))/downsampling_factor
        idx_1 = jnp.array( jnp.floor(idx_n_weight), dtype=int)
        idx_2 = jnp.array( jnp.ceil(idx_n_weight), dtype=int)
        weight_2 = idx_n_weight - idx_1

        state_before = log_state_base_downsampled[idx_1]
        state_after = log_state_base_downsampled[idx_2]
        return (1-weight_2)*state_before + weight_2*state_after
        
    def sample_wday_effect(contrasts_wday, wday):
        # Prior for day-of-the-week effects (6 coefficients)
        wday_coefficients = numpyro.sample("wday_coefficients", dist.Normal(0, 1), sample_shape=(6,))

        # Compute wday effect per observation (sum-to-zero constraint applied via contrasts)
        wday_effects = jnp.dot(contrasts_wday, wday_coefficients)
        return jnp.array([wday_effects[d - 1] for d in wday]) # to-do: just use an index vector instead of a loop

    def sample_yday_effect(contrasts_yday, yday_fraction):
        # Prior for yearly seasonality effects (12 coefficients)
        yday_coefficients = numpyro.sample("yday_coefficients", dist.Normal(0, 1), sample_shape=(12,))
        return jnp.dot(contrasts_yday, yday_coefficients)

    def sample_price_effect(log_price_centered):
        # Prior for price elasticity
        log_elasticity = numpyro.sample( "log_elasticity", dist.HalfNormal(0, 1.5) )
        elasticity = numpyro.deterministic( "elasticity", -1 * jnp.exp( log_elasticity ))
        return log_price_centered * elasticity


    # Sample random walk    
    if n_obs == n_states:
        log_state_base = sample_random_walk(contrasts_sdif_t, n_states)
    else:
        log_state_base = sample_downsampled_random_walk(contrasts_sdif_t, n_obs, n_states)

    # Sample day-of-the-week effects
    wday_effect = sample_wday_effect(contrasts_wday, wday)

    # Sample day-of-the-year effects
    yday_effect = sample_yday_effect(contrasts_yday, yday_fraction)

    # Sample elasticity effect
    price_effect = sample_price_effect(log_price_centered)

    # Compute state
    state = numpyro.deterministic("state", jnp.exp(log_state_base + price_effect + yday_effect + wday_effect))

    # Compute log-likelihood for poisson emissions
    numpyro.sample("sales", dist.Poisson(rate=state), obs=sales)

### 1.3 Model Fitting Logic

- Functions to fit the model

In [26]:
def prepare_model_arguments(sales: jnp.array, log_price: jnp.array, wday, yday_fraction: jnp.array, downsampling_factor = 1):
    """ 
    """    
    n_obs = len(sales)
    if downsampling_factor == 1:
        n_states = n_obs
    else:
        n_states = int( np.floor(n_obs/downsampling_factor) + 1 ) 
    
    # Define contrast matrix for random walk (T coefficients, sum-to-zero constraint)
    contrasts_sdif_t = patsy.contrasts.Diff().code_without_intercept(range(0, n_states)).matrix

    # Define contrast matrix for day-of-the-week effects (6 coefficients, sum-to-zero constraint)
    contrasts_wday = patsy.contrasts.Diff().code_without_intercept(range(0,7)).matrix  # 7 days → 6 contrasts

    # Compute yday effect per observation (sum-to-zero constraint applied via contrasts)
    contrasts_yday = compute_doy_basis(yday_fraction, sigma = 30/365.25, n_centers = 12)

    # Compute centered log price differences
    log_price_centered = log_price - jnp.mean(log_price)

    # Set up the model parameters
    model_arguments = {'sales': sales, 'log_price_centered': log_price_centered, 'wday': wday, 'yday_fraction': yday_fraction,
                       'downsampling_factor': downsampling_factor,
                       'contrasts_sdif_t': contrasts_sdif_t, 'contrasts_wday': contrasts_wday, 'contrasts_yday': contrasts_yday}
    
    # Prepare init values for parameters 
    init_params = init_values(sales, log_price_centered, wday, yday_fraction)

    return init_params, model_arguments

In [ ]:
def run_nuts(sales: jnp.array, log_price: jnp.array, wday, yday_fraction: jnp.array, downsampling_factor = 1, n_chains = 1, num_warmup=1_000, num_samples=1_000):
    """ Runs NUTS MCMC inference on the model 
    """
    rng_key = random.PRNGKey(0)
    
    n_obs = len(sales)
    
    # Prepare model arguments
    init_params, model_arguments = prepare_model_arguments(sales = sales, log_price = log_price, wday = wday, yday_fraction = yday_fraction, downsampling_factor = downsampling_factor)

    rng_key, rng_key_ = random.split(rng_key)

    numpyro.set_host_device_count(n_chains)

    reparam_model = model_local_level_poisson
    kernel = NUTS(reparam_model, step_size=0.01, max_tree_depth=8)
    mcmc = MCMC(kernel, num_warmup=num_warmup, num_samples=num_samples, num_chains=n_chains)
    mcmc.run( rng_key_, **model_arguments) #, init_params = init_params

    return mcmc  

## 2. Fit the Model 

In [29]:

data = read_data("sales_synthetic.csv")


In [ ]:
m_fit = run_nuts(data['sales'], data['log_price'], data['wday'], data['yday_fraction'], downsampling_factor = 7, n_chains = 4, num_warmup=1_000, num_samples=1_000)

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
summary = az.summary(m_fit)

In [16]:
#import arviz as az
#summary = az.summary(m['mcmc'])
summary.loc[['sigma', 'elasticity']]

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
sigma,0.069,0.007,0.057,0.083,0.000,0.000,1000.0,1964.0,1.0
elasticity,-0.550,0.265,-1.026,-0.100,0.006,0.004,1523.0,1478.0,1.0


In [17]:
summary

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
elasticity,-0.550,0.265,-1.026,-0.100,0.006,0.004,1523.0,1478.0,1.00
log_elasticity,-0.733,0.560,-1.727,0.235,0.015,0.014,1523.0,1478.0,1.00
log_sigma,-2.677,0.102,-2.868,-2.493,0.003,0.001,1000.0,1964.0,1.00
log_state_base[0],2.246,0.119,2.033,2.473,0.003,0.002,2231.0,2755.0,1.00
log_state_base[1],2.273,0.107,2.078,2.482,0.002,0.002,1992.0,2649.0,1.00
...,...,...,...,...,...,...,...,...,...
yday_coefficients[7],-0.115,0.379,-0.844,0.609,0.019,0.010,416.0,762.0,1.00
yday_coefficients[8],0.167,0.374,-0.469,0.915,0.019,0.009,392.0,819.0,1.01
yday_coefficients[9],-0.205,0.383,-0.951,0.502,0.019,0.011,398.0,810.0,1.00
yday_coefficients[10],0.003,0.368,-0.681,0.693,0.019,0.009,387.0,759.0,1.01


In [ ]:
az.summary(data=idata_parameters, var_names=["log_sigma", "log_price_coefficient", "yday_coefficients", "wday_coefficients"], round_to=3)

In [ ]:
import inspect
print(inspect.getsource(m['mcmc'].print_summary))

In [ ]:
import jax
from datetime import date
rng_key = jax.random.key(int(date.today().strftime("%Y%m%d")))

import numpy as np
import jax.numpy as jnp
from jax.scipy.stats import multivariate_normal


In [ ]:
import jax
from datetime import date
rng_key = jax.random.key(int(date.today().strftime("%Y%m%d")))

In [ ]:
import numpy as np
import jax.numpy as jnp
from jax.scipy.stats import multivariate_normal

import blackjax
import blackjax.smc.resampling as resampling
from blackjax.smc import extend_params

from numpyro.infer.util import initialize_model

In [ ]:
init_params, model_arguments = prepare_model_arguments(sales, log_price, wday, yday_fraction)    

rng_key, init_key = jax.random.split(rng_key)
init_params, potential_fn_gen, *_ = initialize_model(
    init_key,
    model_local_level_poisson,
    model_kwargs=model_arguments,
    dynamic_args=True,
)

In [ ]:
logdensity_fn = lambda position: -potential_fn_gen(**model_arguments)(position)
initial_position = init_params.z

In [ ]:
import blackjax

num_warmup = 2000

adapt = blackjax.window_adaptation(
    blackjax.nuts, logdensity_fn, target_acceptance_rate=0.8
)
rng_key, warmup_key = jax.random.split(rng_key)
(last_state, parameters), _ = adapt.run(warmup_key, initial_position, num_warmup)
kernel = blackjax.nuts(logdensity_fn, **parameters).step

In [ ]:
def inference_loop(rng_key, kernel, initial_state, num_samples):
    @jax.jit
    def one_step(state, rng_key):
        state, info = kernel(rng_key, state)
        return state, (state, info)

    keys = jax.random.split(rng_key, num_samples)
    _, (states, infos) = jax.lax.scan(one_step, initial_state, keys)

    return states, (
        infos.acceptance_rate,
        infos.is_divergent,
        infos.num_integration_steps,
    )

In [ ]:
num_sample = 1000
rng_key, sample_key = jax.random.split(rng_key)
states, infos = inference_loop(sample_key, kernel, last_state, num_sample)
#_ = states.position["mu"].block_until_ready()

In [ ]:
states

In [ ]:
svi_result, posterior_parameters, posterior_generated = run_svi(sales, log_price, wday, yday_fraction)

In [ ]:
idata_parameters = az.from_dict(
     posterior = { k: np.expand_dims(a=np.asarray(v), axis=0) for k, v in posterior_parameters.items() },
 )
idata_generated = az.from_dict(
     posterior={ k: np.expand_dims(a=np.asarray(v), axis=0) for k, v in posterior_generated.items() },
 )

In [ ]:
#az.summary(data=idata_generated, var_names=["sigma"], round_to=3)
az.summary(data=idata_parameters, var_names=["log_sigma", "log_price_coefficient", "yday_coefficients", "wday_coefficients"], round_to=3)

In [ ]:
az.summary(data=idata_generated, var_names=["state"], round_to=3)

In [ ]:
state = jnp.mean(posterior_generated['state'], axis=0).tolist()
df = df.with_columns([ pl.Series("state", state) ])
df

In [ ]:
posterior_generated

In [ ]:
x = pd.DataFrame({ 'date': df["date"].to_numpy(), 'sales': df["sales"].to_numpy(), 'state': m['state'] })
ggplot(x, aes(x='date', y='sales')) + geom_point() + geom_line(aes(y='state'), color = "red") + theme_bw()